In [207]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
default = {'scenario': 'base','training_folder':'../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

In [208]:
import os
import time
import geopandas as gpd
import pandas as pd
import numpy as np
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [209]:
scenario = argv['scenario']


on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)

training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if not on_lambda:
    scenario_folder = training_folder + '/scenarios/' + scenario + '/inputs/'
    output_folder = training_folder + '/scenarios/' + scenario + '/outputs/'
    model_folder = training_folder + '/scenarios/' + scenario + '/model/'
else:
    scenario_folder = input_folder
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/base/outputs/
scen folder :  ../../scenarios/base/inputs/
model folder :  ../../scenarios/base/model/


In [210]:
if 'params' in argv.keys():
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
else:
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)

# inputs

In [211]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')
distribution = stepmodel.read_zippedpickles(model_folder + 'distribution')

sm.volumes = distribution.volumes

zones_before_clustering: 100%|██████████| 2/2 [00:00<00:00, 1296.34it/s]


# od_set

In [212]:
v = sm.volumes.set_index(['origin', 'destination']).sum(axis=1)
od_set = set(v.loc[v>0].index)

In [213]:
cutoff = var["pathfinder"]["cutoff"]

In [214]:
cutoff

50000

In [215]:
help(sm.step_pt_pathfinder)

Help on method step_pt_pathfinder in module quetzal.model.transportmodel:

step_pt_pathfinder(broken_routes=True, broken_modes=True, route_column='route_id', mode_column='route_type', boarding_time=None, speedup=False, walk_on_road=False, keep_pathfinder=False, force=False, path_analysis=True, **kwargs) method of quetzal.model.stepmodel.StepModel instance
    Performs public transport pathfinder.
    
    With :
        - all or nothing Diskjstra algorithm if broken_routes=False AND broken_modes=False
        - Prunning algorithm if broken_routes=True OR/AND broken_modes=True
    For optimal strategy pathfinder, use step_pt_pathfinder of the class OptimalModel
    
    For connection scan pathfinder algorithm (with time tables), use step_pt_pathfinder of the class ConnectionScanModel.
    
    Requires
    ----------
    self.zones
    self.links
    self.footpaths
    self.zone_to_road
    self.zone_to_transit
    
    Parameters
    ----------
    broken_routes : bool, optional, defa

In [216]:
sm_fast= sm.copy()
sm_fast.links['time'] = sm_fast.links['time']/3

sm_fast.step_pt_pathfinder(
    broken_routes=True,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    cutoff=cutoff,
    verbose=False,
)
sm_fast.pt_los['pathfinder_session']='fast_pt'
#path finder a sm_fast(diviser vitesse par 3,5 ou 10 wtv)
#concacenate sm_fast.pt_los a sm.pt_los
#sm.analysis pt time va remettre les bons temps

start publicpathfinder


route_type {'bus'}: 100%|██████████| 1/1 [00:00<00:00,  8.78it/s]


path_analysis


# PT pathfinder

In [217]:

sm.step_pt_pathfinder(
    broken_routes=True,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    cutoff=cutoff,
    verbose=False,
)
ppf = sm.publicpathfinder

start publicpathfinder


route_type {'bus'}: 100%|██████████| 1/1 [00:00<00:00,  8.89it/s]


path_analysis


In [218]:
sm.pt_los = pd.concat([sm.pt_los, sm_fast.pt_los])

In [219]:
sm.road_links['time_ff'] = sm.road_links['length']/(sm.road_links['speed']*1000/3600)
sm.step_road_pathfinder(method='aon', access_time='time',volume_column= 'car_owner',  time_column='time_ff', od_set=od_set,path_analysis=True )

In [220]:
sm.car_los.set_index(['origin', 'destination']).loc['zone_0', 'zone_0']

time              0.0
path         [zone_0]
gtime             0.0
node_path          []
link_path          []
ntlegs             []
Name: (zone_0, zone_0), dtype: object

In [221]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car', 'subway', 'tram','express_bus', 'bus','walk'])

path_analysis: 100%|██████████| 7757/7757 [00:00<00:00, 28738.39it/s]


In [222]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

car_los: 100%|██████████| 29/29 [00:00<00:00, 38.55it/s]            


In [223]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined